In [1]:
import swisseph as swe
from vedicastro.utils import *
from vedicastro import VedicAstro as va

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
def decimal_to_dms(decimal_deg: float):
    signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']
    degrees = int(decimal_deg)
    sign_index = degrees // 30
    degrees_in_sign = degrees % 30
    minutes = int((decimal_deg - degrees) * 60)
    seconds = (decimal_deg - degrees - minutes/60) * 3600.00
    return signs[sign_index], f"{degrees_in_sign:+02}:{minutes:02}:{seconds:02.0f}"

### Common `SideReal` Ayanamsas for Vedic Astrology
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/krishnamurti_ayanamshas.html
- https://astrorigin.com/pyswisseph/sphinx/ephemerides/sidereal/spica_citra_tradition_and_lahiri_ayanamsha.html


In [4]:
SWE_AYANAMSHAS = { "Krishnamurti_Old" : swe.SIDM_KRISHNAMURTI, "Krishnamurti_New": swe.SIDM_KRISHNAMURTI_VP291,
                "Lahiri_1940": swe.SIDM_LAHIRI_1940, "Lahiri_VP285" : swe.SIDM_LAHIRI_VP285, 
                "Lahiri_ICRC" : swe.SIDM_LAHIRI_ICRC,
                }

In [5]:
## Case 1
# year = 2024
# month = 8
# day = 14
# hour = 22
# minute = 31
# secs = 31
# latitude, longitude, utc = 28.6334, 77.2834, "+5:30" ## Delhi

In [6]:
## Case 2
year = 2000
month = 12
day = 30
hour = 22
minute = 31
secs = 59
latitude, longitude, utc_offset = 28.6334, 77.2834, "+5:30" ## Delhi

In [7]:
from pytz import timezone
zone = "Asia/Calcutta"
### TIMEZONE INFORMATION
tz = timezone(zone)
localized_time = tz.localize(datetime(year, month, day, hour, minute, secs))

### Results using native `pyswisseph` library


In [8]:
utc_float =  utc_offset_str_to_float(utc_offset)

utc = swe.utc_time_zone(year, month, day, hour = hour, minutes = minute, seconds = secs, offset = utc_float)
_ , jd_start = swe.utc_to_jd(*utc) ## Unpacks utc tuple
current_time = jd_start

In [9]:
import pandas as pd

# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

results = []  # List to store the results

for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags=swe.FLG_SIDEREAL)
    asc = cusps[0]
    sign, sign_dms = decimal_to_dms(asc)
    results.append({
        'Ayanamsa': ayan,
        'AscSign': sign,
        'AscDecLong': asc,
        'AscDMS': sign_dms
    })

# Convert the list of dictionaries into a pandas DataFrame
df_swisseph = pd.DataFrame(results)
# df_swisseph

### Using the inbuilt `swisseph` function for degree -> DMS conversion

In [10]:
import pandas as pd

# Assuming SWE_AYANAMSHAS and other necessary imports and variables are already defined

results = []  # List to store the results
signs = ['Aries', 'Taurus', 'Gemini', 'Cancer', 'Leo', 'Virgo', 'Libra', 'Scorpio', 'Sagittarius', 'Capricorn', 'Aquarius', 'Pisces']

for ayan in list(SWE_AYANAMSHAS.keys()):
    swe.set_sid_mode(SWE_AYANAMSHAS.get(ayan))  # Set the ayanamsa based on the current key
    cusps, asmc = swe.houses_ex(current_time, latitude, longitude, b'P', flags=swe.FLG_SIDEREAL)
    asc = cusps[0]
    flag = swe.SPLIT_DEG_ZODIACAL 
    deg, mins, sec, fractional_sec, zodiac = swe.split_deg(asc, flag)
    sign = signs[zodiac] 
    sign_dms = f"{deg:+02}:{mins:02}:{sec:02.0f}"
    
    results.append({
        'Ayanamsa': ayan,
        'AscSign': sign,
        'AscDecLong': asc,
        'AscDMS': sign_dms
    })

# Convert the list of dictionaries into a pandas DataFrame
df_swisseph2 = pd.DataFrame(results)
# df_swisseph

### Results from `VedicAstro` which uses `flatlib`

In [11]:
from flatlib import const

flatlib_ayanamshas =  ["Krishnamurti", "Lahiri"]
house_system = "Placidus"

flatlib_results = []
for ayan in flatlib_ayanamshas:
    vhd = va.VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc_offset, 
                            latitude = latitude, longitude = longitude, ayanamsa = ayan, house_system = house_system)
    chart = vhd.generate_chart()
    asc = chart.get(const.ASC)
    asc_sign, asc_dms = str(asc).split()[1:]
    flatlib_results.append({"Ayanamsa" : ayan, "AscSign": asc.sign, "AscDecLong": asc.lon, "AscDMS" : asc_dms[:-1]})

In [12]:
df_flatlib = pd.DataFrame(flatlib_results)
# df_flatlib

In [13]:
# Convert DataFrames to HTML and concatenate them with some space in between
df1_html = df_swisseph.to_html()
df2_html = df_flatlib.to_html()
df3_html = df_swisseph2.to_html()

# Titles for each DataFrame
title1 = "Swisseph - Asc Results"
title2 = "Flatlib - Asc Results"
title3 = "Swisseph (new dms function) - Asc"

# Display them side by side with titles using HTML display with formatted string
html = f"""
<div style='display:flex; justify-content:space-between; flex-direction: column'>
    <div style='display:flex;'>
        <div style='width: 50%; padding-right: 10px;padding-left: 10px'>
            <h2>{title1}</h2>  <!-- Title for the first DataFrame -->
            {df1_html}
        </div>
        <div style='width: 50%; padding-right: 10px;padding-left: 10px'>
            <h2>{title3}</h2>  <!-- Title for the third DataFrame -->
            {df3_html}
        </div>
    </div>
    <div style='width: 100%;'>
        <h2>{title2}</h2>  <!-- Title for the second DataFrame -->
        {df2_html}
    </div>
</div>
"""
print(f"Asc Results for {localized_time} at Lat:{latitude} | Long: {longitude}")
display(HTML(html))

Asc Results for 2000-12-30 22:31:59+05:30 at Lat:28.6334 | Long: 77.2834


,Ayanamsa,AscSign,AscDecLong,AscDMS
0,Krishnamurti_Old,Leo,140.590161,+20:35:25
1,Krishnamurti_New,Leo,140.570036,+20:34:12
2,Lahiri_1940,Leo,140.508078,+20:30:29
3,Lahiri_VP285,Leo,140.486920,+20:29:13
4,Lahiri_ICRC,Leo,140.493612,+20:29:37
,Ayanamsa,AscSign,AscDecLong,AscDMS
0,Krishnamurti_Old,Leo,140.590161,+20:35:24
1,Krishnamurti_New,Leo,140.570036,+20:34:12
2,Lahiri_1940,Leo,140.508078,+20:30:29
3,Lahiri_VP285,Leo,140.486920,+20:29:12
